In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.stats import uniform
from scipy.stats import norm
from scipy.stats import expon
from scipy.optimize import minimize
from mpl_toolkits import mplot3d
%matplotlib inline

In [2]:
def Z(mu, sigma):
    return(norm.cdf(mu/sigma))

def g(x, mu, sigma):
    return((x > 0)*norm.pdf(x, mu, sigma)/(Z(mu, sigma)))

def M1(mu, sigma):
    return(1/Z(mu, sigma))

def Lambda_Star(mu, sigma):
    Delta = mu*mu+4*sigma*sigma
    return((-mu+np.sqrt(Delta))/(2*sigma*sigma))

def M2(mu, sigma):
    L_Star = Lambda_Star(mu, sigma)
    return((np.exp(mu*L_Star+sigma*sigma*L_Star*L_Star/2))/(L_Star*Z(mu, sigma)*np.sqrt(2*np.pi)*sigma))

def AR_Norm_Simple(mu, sigma):
    X = -1
    M_1 = M1(mu, sigma)
    M_2 = M2(mu, sigma)
    if(M_2 >= M_1):
        while(X == -1):
            Y = mu+sigma*norm.rvs()
            if(Y > 0):
                X = Y
    else:
        while(X == -1):
            L_Star = Lambda_Star(mu, sigma)
            Y = expon.rvs(scale = 1/L_Star)
            U = uniform.rvs()
            if(g(Y, mu, sigma)/(M_2*expon.pdf(Y, scale = 1/L_Star)) >= U):
                X = Y
    return(X) 

def params(i, mu, Q, X):
    mu_x = mu[i]-np.sum([Q[i,j]*(X[j]-mu[j]) for j in range(len(mu)) if j != i])/Q[i,i]
    sigma_squared_x = 1/Q[i,i]
    return(mu_x, sigma_squared_x)

def Gibbs_Sampler(n, mu, Sigma, X0):
    d = len(mu)
    Q = np.linalg.inv(Sigma)
    X = [list(X0)]
    for _n in range(n-1):
        for i in range(d):
            mu_x, s_s_x = params(i, mu, Q, X0)
            X0[i] = AR_Norm_Simple(mu_x, np.sqrt(s_s_x))
        X.append(list(X0))
    return(np.array(X)[int(0.05*n):,:])

def plot_Gibbs(X, mu, Sigma):
    d = len(mu)
    fig, axs = plt.subplots(d, figsize = (10,15))
    x = np.linspace(0, 50, 10000)
    for t in range(d):
        y = norm.pdf(x, loc = mu[t], scale = np.sqrt(Sigma[t,t]))/Z(mu[t], np.sqrt(Sigma[t,t]))
        axs[t].hist(X[:,t], density = True, bins = 40, label = "Histogramme des simulations")
        axs[t].plot(x, y, label = "Densité théorique")
    fig.suptitle('Densités théoriques et pratiques', y = 0.9)
    
def autocorr(X, t):
    X_t = np.transpose(X)
    if(t == 0):
        return(np.corrcoef(X_t[:,:],X_t[:,:]))
    return(np.corrcoef(X_t[:,t:],X_t[:,:-t]))

def graph_autocorr(X, t = 50):
    Lags = [_n for _n in range(t+1)]
    d = X.shape[1]
    fig, axs = plt.subplots(d, figsize = (10,15))
    for _d in range(d):
        Auto = [autocorr(X, lag)[_d,_d+3] for lag in Lags]
        axs[_d].plot([-int(t/50),t+int(t/50)],[0,0], color = 'black')
        axs[_d].plot([-int(t/50),t+int(t/50)],[0.2,0.2], color = 'blue', linestyle='dashed')
        axs[_d].plot([-int(t/50),t+int(t/50)],[-0.2,-0.2], color = 'blue', linestyle='dashed')
        for i in range(len(Auto)):
            axs[_d].plot([Lags[i], Lags[i]], [0, Auto[i]], color = 'black')
        axs[_d].axis([-int(t/50),t+int(t/50),min(Auto)-0.1,max(Auto)+0.1])
    fig.suptitle('Graphes des autocorrélations', y = 0.9)

In [3]:
#d = 3
#n = 10526
#mu = np.array([3.,0.,5.])
#Sigma = np.array([[3.,0.,1.],
#                  [0.,4.,0.2],
#                  [1.,0.2,5.]])
#Q = np.linalg.inv(Sigma)
#X0 = expon.rvs(size = 3)
#X = Gibbs_Sampler(n, mu, Sigma, X0)
#plot_Gibbs(X, mu, Sigma)
#graph_autocorr(X)

n = 5_000
d = 2
mu = np.array([3,0])
Sigma = np.array([[2,0.3],
              [0.3,4]])
Q = np.linalg.inv(Sigma)

X0 = expon.rvs(size = 2)

X = Gibbs_Sampler(n, mu, Sigma, X0)

In [4]:
#m = n
#mu_prime = np.array([3.2,-0.4,5.5])
#Sigma_prime = np.array([[2.6,0.4,1.2],
#                        [0.4,3.5,0.1],
#                        [1.2,0.1,4.3]])
#Q_prime = np.linalg.inv(Sigma_prime)
#Y0 = expon.rvs(size = 3)
#Y = Gibbs_Sampler(n, mu_prime, Sigma_prime, Y0)
#plot_Gibbs(Y, mu_prime, Sigma_prime)
#graph_autocorr(Y)
#
N = 50_000

mu_prime = np.array([3.2,-0.2])
Sigma_prime = np.array([[2.5,0.1],
                        [0.1,4.5]])
Q_prime = np.linalg.inv(Sigma_prime)

Y0 = expon.rvs(size = 2)

Y = Gibbs_Sampler(N, mu_prime, Sigma_prime, Y0)

$l(x; \mu; \Sigma) = \log(\dfrac{1}
{Z(\boldsymbol{\mu}, \boldsymbol{\Sigma})(2\pi)^{N/2} \left| \boldsymbol{\Sigma}\right|^{1/2}}\exp\left[
-\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)\right] \, \mathbb{1}_{x > 0})$

$l(x; \mu; \Sigma) = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(Z(\boldsymbol{\mu}, \boldsymbol{\Sigma}))-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst$

$l(x; \mu; \Sigma) = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\dfrac{Z(\boldsymbol{\mu}, \boldsymbol{\Sigma})}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'}))-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst$

or $\dfrac{Z(\boldsymbol{\mu}, \boldsymbol{\Sigma})}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})} = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} \dfrac{1}
{(2\pi)^{N/2} \left| \boldsymbol{\Sigma}\right|^{1/2}}\;\exp\left[
-\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)\right] \, \mathbb{1}_{x > 0}\mathrm{d}x = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} 
f(x; \boldsymbol{\mu}, \boldsymbol{\Sigma}) \mathbb{1}_{x > 0}\mathrm{d}x$

$ = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} 
\dfrac{f(x; \boldsymbol{\mu}, \boldsymbol{\Sigma})}{f(x; \boldsymbol{\mu'}, \boldsymbol{\Sigma'})}f(x; \boldsymbol{\mu'}, \boldsymbol{\Sigma'}) \mathbb{1}_{x > 0}\mathrm{d}x = \mathbb{E}\left[\dfrac{f(Y; \boldsymbol{\mu}, \boldsymbol{\Sigma})}{f(Y; \boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\right]$ où $Y \sim \mathcal{N}_{tr}(\mu', \Sigma')$ où $\mu'$ et $\Sigma'$ ne dépendent pas directement de $\mu$ et $\Sigma$ et sont donc constants d'où:

$l(x; \mu; \Sigma) = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\mathbb{E}\left[\dfrac{f(Y; \boldsymbol{\mu}, \boldsymbol{\Sigma})}{f(Y; \boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\right])-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst $

$ = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\dfrac{\left| \boldsymbol{\Sigma'}\right|^{1/2}}{\left| \boldsymbol{\Sigma}\right|^{1/2}}\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right])-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|) + Cst $

$ = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right])-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|)+\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma}\right|)-\dfrac{1}{2}\log(\left| \boldsymbol{\Sigma'}\right|) + Cst = -\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right) - \log(\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right]) + Cst$

On approximera ensuite $\mathbb{E}\left[\exp\left[\dfrac{\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y}-\boldsymbol{\mu}\right)}{2}\right]\right]$ par: $\dfrac{1}{m}\displaystyle \sum_{i = 1}^{m} \exp\left[\dfrac{\left(\boldsymbol{Y_i}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_i}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_i}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_i}-\boldsymbol{\mu}\right)}{2}\right]$

Finalement on cherche à minimiser:

$L(\mu, \Sigma) = \displaystyle \sum_{i = 1}^{n} \dfrac{1}{2}\left(\boldsymbol{X_i}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{X_i}-\boldsymbol{\mu}\right) + \log(\displaystyle \sum_{j = 1}^{m} \exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right])$, on a, en particulier, les dérivées suivantes:

$\dfrac{\partial L}{\partial \mu}(\mu, \Sigma) = \displaystyle \sum_{i = 1}^{n} - \left(\boldsymbol{X_i}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1} + \dfrac{\displaystyle \sum_{j = 1}^{m} \left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right]}{\displaystyle \sum_{j = 1}^{m} \exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right]}$

$\dfrac{\partial L}{\partial \sigma_{ij}}(\mu, \Sigma) = \displaystyle \sum_{i = 1}^{n} -\dfrac{1}{2}\left(\boldsymbol{X_i}-\boldsymbol{\mu}\right)^\top C(\boldsymbol{\Sigma})\left(\boldsymbol{X_i}-\boldsymbol{\mu}\right) + \dfrac{\displaystyle \sum_{j = 1}^{m} \dfrac{1}{2}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top C(\boldsymbol{\Sigma})\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)\exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right]}{\displaystyle \sum_{j = 1}^{m} \exp\left[\dfrac{\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)^\top\boldsymbol{\Sigma'}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu'}\right)-\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{Y_j}-\boldsymbol{\mu}\right)}{2}\right]}$ 

où $C(\boldsymbol{\Sigma}) =  \boldsymbol{\Sigma}^{-1}\dfrac{\partial \boldsymbol{\Sigma}}{\partial \sigma_{ij}}\boldsymbol{\Sigma}^{-1}$.

In [5]:
def OP(Y, mu, Sigma_inv):
    return(np.dot(np.dot(Y-mu, Sigma_inv), np.transpose(Y-mu))/2)

Stock = np.array([OP(Yi, mu_prime, Q_prime) for Yi in Y])

In [51]:
def Transform(P, d):
    M = np.zeros((d, d))
    for i in range(d):
        M[i, :i+1] = P[int(i*(i+1)/2): int(i*(i+1)/2)+i+1]
    return(M)

def Transform_inv(M):
    return(list(np.concatenate([M[i, :i+1] for i in range(M.shape[0])])))

In [52]:
P = [1,2,3,4,5,6]
Transform_inv(Transfor(P, 3))

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

In [64]:
K = np.linalg.cholesky(Sigma_0)

In [66]:
Transform_inv(K)

[1.4142135623730951, 0.0, 1.4142135623730951]

In [35]:
Test = np.zeros((2,2))
Test[0,:1]

array([0.])

In [62]:
def l(P, X, Y, Stock):
    mu = P[:d]
    K = Transform(P[d:], d)
    Sigma = np.dot(K, K.T)
    Q = np.linalg.inv(Sigma)
    return(np.sum(np.dot(X-mu, Q)*(X-mu))/2+X.shape[0]*np.log(np.sum(np.exp(Stock-np.sum(np.dot(Y-mu, Q)*(Y-mu), axis = 1)/2))))

In [ ]:
np.array([-np.dot(np.dot(Yi-mu, Q), np.transpose(Yi-mu))/2 for Yi in Y])

In [19]:
np.sum([OP(Xi, mu, Q) for Xi in X])

4511.545916075813

In [21]:
np.sum(np.exp(Stock-np.sum(np.dot(Y-mu, Q)*(Y-mu), axis = 1)/2))

43433.29047379688

In [23]:
np.sum(np.exp(Stock-np.array([OP(Yi, mu, Q) for Yi in Y])))

43433.29047379688

In [20]:
np.sum(np.dot(X-mu, Q)*(X-mu))/2

4511.545916075812

In [6]:
np.sum(np.exp(Stock-np.sum(np.dot(Y-mu, Q)*(Y-mu), axis = 1)/2))

43433.29047379688

In [8]:
np.sum(np.array([np.dot(np.dot(Xi-mu, Q), np.transpose(Xi-mu))/2 for Xi in X]))

4511.545916075813

In [9]:
-np.sum(np.dot(X-mu, Q)*(X-mu))/2

-4511.545916075812

In [7]:
np.sum(np.exp(Stock+np.array([-np.dot(np.dot(Yi-mu, Q), np.transpose(Yi-mu))/2 for Yi in Y])))

43433.29047379688

In [17]:
np.sum(np.array([-np.dot(np.dot(Yi-mu, Q), np.transpose(Yi-mu))/2 for Yi in Y]))

-52249.11004380018

In [13]:
(Y-mu)

array([[ 1.22264722,  0.35717143],
       [-1.02306381,  0.86137696],
       [ 1.62110636,  0.87001096],
       ...,
       [ 2.02512131,  3.58202772],
       [-0.3148741 ,  0.04598603],
       [ 1.41578068,  0.24304217]])

In [11]:
def dSigma_ij(Sigma, Q, i, j):
    d = Sigma.shape[0]
    dSigma = np.zeros((d,d))
    dSigma[i, j] = 1
    return(np.dot(np.dot(Q, dSigma),Q))

def dl_ij(mu, Sigma, Q, Stock_expY, i, j):
    dS_ij = dSigma_ij(Sigma, Q, i, j)
    Stock_X = np.array([-OP(Xi, mu, dS_ij) for Xi in X])
    Stock_Y = np.array([OP(Yi, mu, dS_ij) for Yi in Y])
    Somme = np.sum(Stock_X+np.sum(Stock_Y*np.exp(Stock_expY))/np.sum(np.exp(Stock_expY)))
    return(Somme)

def dl_mu(mu, Sigma, Q, Stock_expY):
    Stock_X = np.array([-np.dot(Xi-mu, Q) for Xi in X])
    Stock_Y = np.array([np.dot(Yi-mu, Q) for Yi in Y])
    Stock_Y_T = np.array([Stock_Y[i]*np.exp(Stock_expY[i]) for i in range(len(Stock_Y))])
    Somme = np.sum(Stock_X, axis = 0)+len(Stock_X)*np.sum(Stock_Y_T, axis = 0)/np.sum(np.exp(Stock_expY))
    return(Somme)

def Jacob_l(P):
    mu = P[:d]
    Sigma = P[d:].reshape(d, d)
    Q = np.linalg.inv(Sigma)
    Stock_expY = Stock+np.array([-OP(Yi, mu, Q) for Yi in Y])
    result = np.concatenate((dl_mu(mu, Sigma, Q, Stock_expY), [dl_ij(mu, Sigma, Q, Stock_expY, i, j) for j in range(d) for i in range(d)]))
    return(result)

In [12]:
mu_0 = np.array([2,-1])
Sigma_0 = np.array([[1,2],
                    [2,1]])

In [59]:
Sigma_0 = 2*np.eye(2,2)

In [58]:
S_0 = np.eye(2,2)
S_0

array([[1., 0.],
       [0., 1.]])

In [71]:
P0 = np.concatenate((mu_0, Transform_inv(K)))
start = time.time()
res = minimize(l, P0, method = 'BFGS', args = (X, Y, Stock))
end = time.time()
K_find = Transform(res.x[d:], 2)
print(mu)
print(res.x[:d])
print('\n')
print(np.dot(K_find, K_find.T))
print(Sigma)

print("durée = ", end-start, " sec")

[3 0]
[3.01560486 0.13014538]


[[2.021941   0.26647614]
 [0.26647614 3.65809926]]
[[2.  0.3]
 [0.3 4. ]]
durée =  0.5611560344696045  sec


In [70]:
res

      fun: 55234.61286120346
 hess_inv: array([[ 1.02260513e-03, -3.30971120e-04, -1.61337967e-04,
        -5.01452934e-04,  3.05650482e-04],
       [-3.30971120e-04,  3.32836592e-04, -1.37380447e-04,
         1.91360636e-04, -2.10187217e-04],
       [-1.61337967e-04, -1.37380447e-04,  2.39638057e-04,
         1.04410590e-04, -4.21500246e-05],
       [-5.01452934e-04,  1.91360636e-04,  1.04410590e-04,
         3.07466528e-04, -2.50562915e-04],
       [ 3.05650482e-04, -2.10187217e-04, -4.21500246e-05,
        -2.50562915e-04,  4.77479072e-04]])
      jac: array([0.00048828, 0.        , 0.        , 0.00048828, 0.        ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 190
      nit: 19
     njev: 27
   status: 2
  success: False
        x: array([3.01560486, 0.13014538, 1.42194972, 0.18740194, 1.90341267])

In [30]:
l(np.concatenate((mu, Sigma.flatten())), X, Y, Stock)

55236.70798243858

In [179]:
d = 2
n = 10526
mu = np.array([3,0])
Sigma = np.array([[2,0],[0,4]])
Q = np.linalg.inv(Sigma)
X0 = expon.rvs(size = d)
X = Gibbs_Sampler(n, mu, Sigma, X0)

m = n
mu_prime = np.array([6,-1])
Sigma_prime = np.array([[5,0],[0,2]])
Q_prime = np.linalg.inv(Sigma_prime)
Y0 = expon.rvs(size = d)
Y = Gibbs_Sampler(n, mu_prime, Sigma_prime, Y0)

In [150]:
from scipy.stats import multivariate_normal

In [180]:
I = (1-norm.cdf(0,3,np.sqrt(2)))*(1-norm.cdf(0,0,np.sqrt(4)))

In [185]:
LOL = (1-norm.cdf(0,6,np.sqrt(5)))*(1-norm.cdf(0,-1,np.sqrt(2)))

In [181]:
I

0.4915262866188277

In [186]:
Test_Exp = LOL*\
            np.mean([np.exp(OP(Yi, mu_prime, Q_prime)-OP(Yi, mu, Q))*\
                     np.sqrt(np.linalg.det(Sigma_prime))/np.sqrt(np.linalg.det(Sigma)) for Yi in Y])

In [187]:
Test_Exp

0.4641638156569653

$\dfrac{Z(\boldsymbol{\mu}, \boldsymbol{\Sigma})}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})} = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} \dfrac{1}
{(2\pi)^{N/2} \left| \boldsymbol{\Sigma}\right|^{1/2}}\;\exp\left[
-\dfrac{1}{2}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)^\top\boldsymbol{\Sigma}^{-1}\left(\boldsymbol{x}-\boldsymbol{\mu}\right)\right] \, \mathbb{1}_{x > 0}\mathrm{d}x = \dfrac{1}{Z(\boldsymbol{\mu'}, \boldsymbol{\Sigma'})}\int_{}^{} 
f(x; \boldsymbol{\mu}, \boldsymbol{\Sigma}) \mathbb{1}_{x > 0}\mathrm{d}x$